In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [2]:
VGG16 = tf.keras.applications.VGG16(include_top=True, weights='imagenet')
x = VGG16.layers[-2].output
SIMILARITY_MODEL = tf.keras.Model(inputs = VGG16.input, outputs = x)
SIMILARITY_MODEL.trainable = False
SIMILARITY_MODEL.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [3]:
dataset = tfds.load('imagenet_v2', split='all')
print(dataset)

<PrefetchDataset element_spec={'file_name': TensorSpec(shape=(), dtype=tf.string, name=None), 'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


In [4]:
# Helper functions
def image_to_feature_vector(img):
    return np.array(tf.squeeze(SIMILARITY_MODEL(tf.expand_dims(img, 0))))

from numpy.linalg import norm
def similarity(v1, v2):
    v1 /= norm(v1)
    v2 /= norm(v2)
    return 1. - norm(v2 - v1)

In [5]:
result_gaussian_noise_1 = []
result_gaussian_noise_2 = []
result_gaussian_noise_3 = []
result_uniform_noise_1 = []
result_uniform_noise_2 = []
result_uniform_noise_3 = []
result_opposite = []
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
counter = 0

for example in dataset:
    if counter == 1000:
        break
    image = tf.image.resize_with_pad(example["image"], 224, 224)
    # Normalize
    image = tf.math.divide(image, 255.)
    image = tf.math.subtract(image, mean)
    image = tf.math.divide(image, std)

    base = image_to_feature_vector(image)
    gaussian_noise_1 = image_to_feature_vector(image + np.random.normal(0., 0.01, size=image.shape))
    gaussian_noise_2 = image_to_feature_vector(image + np.random.normal(0., 0.05, size=image.shape))
    gaussian_noise_3 = image_to_feature_vector(image + np.random.normal(0., 0.1, size=image.shape))
    uniform_noise_1 = image_to_feature_vector(image + np.random.uniform(low=-0.05, high=0.05, size=image.shape))
    uniform_noise_2 = image_to_feature_vector(image + np.random.uniform(low=-0.1, high=0.1, size=image.shape))
    uniform_noise_3 = image_to_feature_vector(image + np.random.uniform(low=-0.2, high=0.2, size=image.shape))
    opposite = image_to_feature_vector(1 - image)

    result_gaussian_noise_1.append(similarity(base, gaussian_noise_1))
    result_gaussian_noise_2.append(similarity(base, gaussian_noise_2))
    result_gaussian_noise_3.append(similarity(base, gaussian_noise_3))
    result_uniform_noise_1.append(similarity(base, uniform_noise_1))
    result_uniform_noise_2.append(similarity(base, uniform_noise_2))
    result_uniform_noise_3.append(similarity(base, uniform_noise_3))
    result_opposite.append(similarity(base, opposite))

    counter += 1

result_gaussian_noise_1 = np.array(result_gaussian_noise_1)
result_gaussian_noise_2 = np.array(result_gaussian_noise_2)
result_gaussian_noise_3 = np.array(result_gaussian_noise_3)
result_uniform_noise_1 = np.array(result_uniform_noise_1)
result_uniform_noise_2 = np.array(result_uniform_noise_2)
result_uniform_noise_3 = np.array(result_uniform_noise_3)
result_opposite = np.array(result_opposite)

print("N(0, 0.01):", np.mean(result_gaussian_noise_1), np.std(result_gaussian_noise_1))
print("N(0, 0.05):", np.mean(result_gaussian_noise_2), np.std(result_gaussian_noise_2))
print("N(0, 0.1):", np.mean(result_gaussian_noise_3), np.std(result_gaussian_noise_3))
print("U[-0.05, 0.05]:", np.mean(result_uniform_noise_1), np.std(result_uniform_noise_1))
print("U[-0.1, 0.1]:", np.mean(result_uniform_noise_2), np.std(result_uniform_noise_2))
print("U[-0.2, 0.2]:", np.mean(result_uniform_noise_3), np.std(result_uniform_noise_3))
print("Opposite:", np.mean(result_opposite), np.std(result_opposite))

N(0, 0.01): 0.9923350760763279 0.0027573414359894403
N(0, 0.05): 0.9493309781984426 0.016636714970255224
N(0, 0.1): 0.9138069752678275 0.024010484309535816
U[-0.05, 0.05]: 0.9720541157659609 0.009796554531210038
U[-0.1, 0.1]: 0.9417895949799567 0.01857311282785243
U[-0.2, 0.2]: 0.9068605818971992 0.024809241848446253
Opposite: 0.7913540313020349 0.04939184221451637
